In [1]:
import pandas as pd
import numpy as np
import string
import nltk

In [2]:
#import csv and call reviews in pd
df = pd.read_csv("../fake reviews dataset.csv")

In [3]:
#Text Normalization --> make lower case , remove punctuations , special character, numbers

df['text_'] =df['text_'].str.lower()


#incase of web scrapping will need to remove html tags code for that
# import re
# def remove_html_tags(text):
#   pattern = re.compile('<.*?>')
#   return pattern.sub(r'' , text)
# df['text_'] = df['text_'].apply(remove_html_tags)

exclude = string.punctuation

# def remove_punc(text):
#   for char in exclude :
#     text = text.replace(char, '')
#   return text
#very slow

def remove_punc(text):
  return text.translate(str.maketrans('', '' , exclude))


df['text_'] = df['text_'].apply(remove_punc)

#spelling correction
# from textblob import TextBlob
# textBlb = TextBlob(df['text_'])
# df['text_'] = textBlb.correct().string


In [4]:
#stopword removal --> eliminating common words

# print(stopwords.words('english'))
from nltk.corpus import stopwords
# nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# print(stopwords)
# def remove_stopwords(text):

#   new_text = []

#   for word in text.split():
#     if word in stopwords.words('english'):
#       new_text.append('')
#     else :
#       new_text.append(word)
#   x = new_text[:]
#   new_text.clear()
#   return " ".join(x)

def remove_stopwords(text):
    # Replace stopwords with an empty string but preserve spaces
    return " ".join([word if word not in stop_words else '' for word in text.split()])



df['text_'] = df['text_'].apply(remove_stopwords)

In [5]:
#tokenization --> break the review in words
# import spacy
# nlp = spacy.load('en_core_web_sm')


# def give_tokens(text):
#   tokens = []
#   doc = nlp(text)
#   for token in doc:
#     tokens.append(token)
#   return doc

# df['text_'] = df['text_'].apply(give_tokens)

# df.head()


from nltk.tokenize import word_tokenize, sent_tokenize

def give_token(text):
  return word_tokenize(text)

# str1 = "my name is rhythm"
# word_tokenize(str1)

df['text_'] = df['text_'].apply(give_token)

In [6]:
# #stemming --> eg running -> run
# from nltk.stem.porter import PorterStemmer
# ps = PorterStemmer()

# def stem_words(wordlist):
#   text = []
#   for word in wordlist :
#     text.append(ps.stem(word))
#   return text

# df['text_'] = df['text_'].apply(stem_words)
# df.head()
# # def give(text):
# #   return ps.stem(text)

# # give("walking")


from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def lemmatize_words(text):
    return [wordnet_lemmatizer.lemmatize(word) for word in text]

df['text_'] = df['text_'].apply(lemmatize_words)


In [7]:
# import gensim
# from gensim.models import Word2Vec,KeyedVectors
# model = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin',binary=True,limit=500000)
import pickle 
with open("../vectorize.pkl", "rb") as file:
    model = pickle.load(file)

In [8]:
x = df['text_']
for i in range(len(x)):
    if( len(x[i]) == 0):
        x[i] = ['x']



def get_vector(text):
    return model.get_mean_vector(text)

df['text_'] = df['text_'].apply(get_vector)

x = np.vstack(df['text_'].values)
y = np.array(df['label'].values)


C:\Users\Admin\AppData\Local\Temp\ipykernel_16884\2399057821.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = ['x']


In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#   

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.25, random_state=42)

# Creating an SVM classifier
clf = svm.SVC(kernel='rbf', C=25 )  # You can use 'rbf', 'poly', or 'sigmoid' kernels as well

# Training the classifier
clf.fit(X_train, y_train)


# Making predictions
y_pred = clf.predict(X_test)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

In [ ]:
import pickle 

with open("../review_model.pkl", "wb") as file: 
    pickle.dump(clf, file)
    